In [1]:
from __future__ import division
import csv
import pandas as pd
import requests
import json
import glob
import numpy as np
import pickle
import editdistance

In [2]:
import sys
sys.path.append("..")
from all_functions import *

## combine csvs for titles and WD ids 

In [54]:
path =r'..\OUTPUTS_titles_word_count' # use your path
allFiles = glob.glob(path + "\*.csv")
list_ = []
final_all_titles = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    final_all_titles.append(df)
final_all_titles_df, list_=combine_dataframes_from_batch(final_all_titles,list_)

In [55]:
path =r'..\OUTPUT_wd_id' # use your path
allFiles = glob.glob(path + "\*.csv")
list_ = []
final_all_concord = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    final_all_concord.append(df)
final_all_concord_df, list_=combine_dataframes_from_batch(final_all_concord,list_)

In [56]:
final_all_concord_df['wk:page']=final_all_concord_df['wk_name']
final_all_concord_df['wd:id']=final_all_concord_df['wiki_id']
final_all_concord_df=final_all_concord_df[['wk:page','wd:id','wk_page_id']]

In [57]:
final_all_concord_df=final_all_concord_df[final_all_concord_df['wk:page'].notnull()]
final_all_concord_df=final_all_concord_df[final_all_concord_df['wd:id'].notnull()]

In [58]:
all_wof_data=read_data("..\Analyzed_data\\all_meta_data_wof.csv")
all_wof_data=all_wof_data.drop(['Unnamed: 0'],1)
final_all_titles_df=final_all_titles_df.drop(['Unnamed: 0','Unnamed: 0.1'],1)

In [59]:
final_all_titles_df['wof:id']=np.where(final_all_titles_df['wof:id'].isnull(),final_all_titles_df['id'],final_all_titles_df['wof:id'])

In [60]:
final_all_titles_wof_join=final_all_titles_df.join(all_wof_data.set_index(['id']), on= 'wof:id', how='left', rsuffix='_right')

In [61]:
final_all_titles_wof_join['name']=np.where(final_all_titles_wof_join['name'].isnull(),final_all_titles_wof_join['name_right'],final_all_titles_wof_join['name'])

In [62]:
final_all_concord_df=final_all_concord_df[['wd:id','wk:page']]
final_all_titles_wof_join=final_all_titles_wof_join[['wof:id','name','wk:page','wordcount','wk_page_id']]

In [63]:
final_all_concord_df=find_unique(final_all_concord_df,'wd:id')

In [64]:
final_all_titles_ids_join=final_all_titles_wof_join.join(final_all_concord_df.set_index(['wk:page']), on= 'wk:page', how='left')

In [65]:
final_all_titles_ids_join_not_null=final_all_titles_ids_join[(final_all_titles_ids_join['wk:page'].notnull())]

In [66]:
final_all_titles_ids_join_not_null=find_unique(final_all_titles_ids_join_not_null, 'wof:id')
final_all_titles_ids_join_not_null['name']=np.where(final_all_titles_ids_join_not_null['name'].isnull(),final_all_titles_ids_join_not_null['wk:page'],final_all_titles_ids_join_not_null['name'])

In [67]:
final_all_titles_ids_join_not_null['spell_check']='maybe'

In [68]:
final_all_titles_ids_join_not_null['wof:name']=final_all_titles_ids_join_not_null['name']
final_all_titles_ids_join_not_null.head()

,wof:id,name,wk:page,wordcount,wk_page_id,wd:id,spell_check,wof:name
0,102191569,Asia,Asia,7791,NaN,Q48,maybe,Asia
1,102191573,Africa,Africa,9840,NaN,Q15,maybe,Africa
2,102191575,North America,North America,9285,NaN,Q49,maybe,North America
3,102191577,South America,South America,5946,NaN,Q18,maybe,South America
4,102191579,Antarctica,Antarctica,11937,NaN,Q51,maybe,Antarctica


## Data Clean up process 

#### Find region and country of each entry 

In [69]:
all_wof_data=all_wof_data[['country_id','region_id','placetype','id']]
unique_concordances_dist_wof=final_all_titles_ids_join_not_null.join(all_wof_data.set_index(['id']), on = 'wof:id', how = 'left', rsuffix='_right')
countries=read_data("..\Analyzed_data\\wof-country-latest.csv")
regions=read_data("..\Analyzed_data\\wof-region-latest.csv")
countries=countries[['id', 'name']]
regions = regions[['id', 'name']]

In [70]:
unique_concordances_dist_wof_countries=unique_concordances_dist_wof.join(countries.set_index(['id']), on = 'country_id', how = 'left', rsuffix='_country')
unique_concordances_dist_wof_coun_reg=unique_concordances_dist_wof_countries.join(regions.set_index(['id']), on = 'region_id', how = 'left', rsuffix='_region')
unique_concordances_dist_wof_coun_reg['new_name_country'] = unique_concordances_dist_wof_coun_reg['name']+', '+unique_concordances_dist_wof_coun_reg['name_country']
unique_concordances_dist_wof_coun_reg['new_name_region'] = unique_concordances_dist_wof_coun_reg['name']+', '+unique_concordances_dist_wof_coun_reg['name_region']

#### Calculate Levenshtein Distance 

In [71]:
unique_concordances_dist=[]
for index, row in unique_concordances_dist_wof_coun_reg.iterrows():
    leve_dist=editdistance.eval(row['name'], row['wk:page'])
    length1=len(row['wk:page'])
    length2=len(row['name'])
    rank=leve_dist/max(length1,length2)
    row['leve_dist']=rank
    unique_concordances_dist.append(row)

unique_concordances_dist=pd.DataFrame(unique_concordances_dist)

In [72]:
unique_concordances_dist_country=[]
for index, row in unique_concordances_dist.iterrows(): 
    if row['leve_dist'] == 0.0:
        row['leve_country'] = row['leve_dist']
    elif str(row['name_country']) == 'nan':  
        row['leve_country'] = row['leve_dist']
    else:
        leve_dist=editdistance.eval(row['new_name_country'], row['wk:page'])
        length1=len(row['wk:page'])
        length2=len(row['new_name_country'])
        rank=leve_dist/max(length1,length2)
        row['leve_country']=rank
    unique_concordances_dist_country.append(row)
unique_concordances_dist_country=pd.DataFrame(unique_concordances_dist_country)

In [73]:
unique_concordances_dist_count_region=[]
for index, row in unique_concordances_dist_country.iterrows(): 
    if row['leve_country'] == 0.0:
        row['leve_region'] = row['leve_country']
    elif str(row['name_region']) == 'nan':  
        row['leve_region'] = row['leve_country']
    else:
        leve_dist=editdistance.eval(row['new_name_region'], row['wk:page'])
        length1=len(row['wk:page'])
        length2=len(row['new_name_region'])
        rank=leve_dist/max(length1,length2)
        row['leve_region']=rank
    unique_concordances_dist_count_region.append(row)
unique_concordances_dist_count_region=pd.DataFrame(unique_concordances_dist_count_region)

In [74]:
unique_concordances_dist_actual=[]
for index, row in unique_concordances_dist_count_region.iterrows():
    leve_dist=editdistance.eval(row['name'], row['wk:page'])
    rank=leve_dist
    row['leve_dist_actual']=rank
    if str(row['name_country']) != 'nan':
        leve_dist_cou=editdistance.eval(row['new_name_country'], row['wk:page'])
        rank_co=leve_dist_cou
        row['leve_dist_co_actual']=rank_co
    if str(row['name_region']) != 'nan':
        leve_dist_re=editdistance.eval(row['new_name_region'], row['wk:page'])
        rank_re=leve_dist_re
        row['leve_dist_re_actual']=rank_re
    unique_concordances_dist_actual.append(row)
unique_concordances_dist_actual=pd.DataFrame(unique_concordances_dist_actual)

#### If Levenshtein distance matches say OK

In [75]:
unique_concordances_dist_count_region_1=[]
for index, row in unique_concordances_dist_actual.iterrows():
    if row['leve_dist'] == 0.0 or row['leve_country'] == 0.0 or row['leve_region'] == 0.0:
        row['spell_check'] = 'OK'
    unique_concordances_dist_count_region_1.append(row)
unique_concordances_dist_count_region_1=pd.DataFrame(unique_concordances_dist_count_region_1)


#### Clean for Lists, Timelines, Birthdays, numbers etc.. 

In [76]:
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("List")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("Timeline")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("Birthday")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("Aircraft")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("Political")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("Environmental")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("International")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("music")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("History")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("1")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("2")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("3")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("4")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("5")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("6")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("7")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("8")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("9")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_dist_count_region_1.ix[(unique_concordances_dist_count_region_1['wk:page'].str.contains("0")) & (unique_concordances_dist_count_region_1['spell_check']=='maybe'), 'spell_check'] = 'NO'

#### Clean for Airpots and Facilities 

In [77]:
unique_concordances_dist_count_region_2=[]
for index, row in unique_concordances_dist_count_region_1.iterrows():
    if ((row['name'].find('Air')!=-1) and (row['wk:page'].find('Air') == -1)):
        row['spell_check'] = 'NO'
    elif ((row['name'].find('Field')!=-1) and (row['wk:page'].find('Field') == -1)):
        row['spell_check'] = 'NO'
    elif ((row['name'].find('Facility')!=-1) and (row['wk:page'].find('Facility') == -1)):
        row['spell_check'] = 'NO'
    elif ((row['name'].find('Aero')!=-1) and (row['wk:page'].find('Air') == -1)):
        row['spell_check'] = 'NO'
    else:
        pass
    unique_concordances_dist_count_region_2.append(row)
unique_concordances_dist_count_region_2=pd.DataFrame(unique_concordances_dist_count_region_2)

In [78]:
unique_concordances_fix_maybe = unique_concordances_dist_actual

In [79]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Curacao'), 'spell_check'] = 'OK'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='السودان'), 'spell_check'] = 'OK'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Bronx'), 'spell_check'] = 'OK'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Coxs Bazar Airport'), 'spell_check'] = 'OK'

####  Check for Levenshtein distances

In [80]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['leve_dist_actual']<2) & (unique_concordances_fix_maybe['leve_dist_actual']>0)& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'

In [81]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['leve_dist_re_actual']<6) & (unique_concordances_fix_maybe['leve_dist_re_actual']>0)& (unique_concordances_fix_maybe['spell_check']=='maybe') , 'spell_check'] = 'OK'

In [82]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['leve_dist_co_actual']<4) & (unique_concordances_fix_maybe['leve_dist_co_actual']>0) &(unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'

In [83]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['leve_dist']<0.4)& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'

In [84]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['leve_region']<0.3)& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'

In [85]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['leve_country']<0.3)& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'

In [86]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['leve_dist']>0.8)& (unique_concordances_fix_maybe['spell_check']=='maybe')&(unique_concordances_fix_maybe['name_country']!='Greece')&(unique_concordances_fix_maybe['name_country']!='Cyprus'), 'spell_check'] = 'NO'

In [87]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['leve_region']>0.8)& (unique_concordances_fix_maybe['spell_check']=='maybe')&(unique_concordances_fix_maybe['name_country']!='Greece')&(unique_concordances_fix_maybe['name_country']!='Cyprus'), 'spell_check'] = 'NO'

In [88]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['leve_country']>0.8)& (unique_concordances_fix_maybe['spell_check']=='maybe')&(unique_concordances_fix_maybe['name_country']!='Greece')&(unique_concordances_fix_maybe['name_country']!='Cyprus'), 'spell_check'] = 'NO'

In [89]:
print_test_leve=unique_concordances_fix_maybe[((unique_concordances_fix_maybe['name']=='In Amguel Airport')|
                                   (unique_concordances_fix_maybe['name']=='Scottsville')|
                                   (unique_concordances_fix_maybe['name']=='Lewis B Wilson Airport')|
                                   (unique_concordances_fix_maybe['name']=='Pontiac Municipal Airport')|
                                     (unique_concordances_fix_maybe['name']=='Tiska Airport')|
                                     (unique_concordances_fix_maybe['name']=='Coxs Bazar Airport')|
                                   (unique_concordances_fix_maybe['name']=='Ishurdi Airport')|
                                   (unique_concordances_fix_maybe['name']=='Tafaraoui Airport'))]
print_test_leve=print_test_leve[(print_test_leve['wk:page']!='Scottsville')]
print_test_leve=print_test_leve[(print_test_leve['wk:page']!='Scottsville, Virginia')]
print_test_leve=print_test_leve[(print_test_leve['wk:page']!='Scottsville, Texas')]
print_test_leve=print_test_leve[(print_test_leve['wk:page']!='Scottsville, New York')]
print_test_leve=print_test_leve[(print_test_leve['wof:id']!=102521985)]
print_test_leve['leve_dist_r'] = print_test_leve['leve_dist'].round(2)
print_test_leve['leve_region_r'] = print_test_leve['leve_region'].round(2)
print_test_leve['leve_country_r'] = print_test_leve['leve_country'].round(2)
show_data_1=print_test_leve[['wof:name', 'wk:page','name_region','leve_dist_r','leve_region_r','leve_country_r','spell_check']]
show_data_1

,wof:name,wk:page,name_region,leve_dist_r,leve_region_r,leve_country_r,spell_check
28,Coxs Bazar Airport,Cox's Bazar Airport,Chittagong,0.05,0.43,0.43,OK
31,Ishurdi Airport,Ishwardi Airport,Rajshahi,0.12,0.48,0.52,OK
39,Tafaraoui Airport,Oran Tafraoui Airport,Oran,0.29,0.52,0.58,OK
65,In Amguel Airport,Tamanrasset,Tamanghasset,0.88,0.68,0.88,NO
73,Tiska Airport,Djanet Inedbirene Airport,Illizi,0.64,0.88,0.84,NO
109,Lewis B Wilson Airport,Middle Georgia Regional Airport,Georgia,0.61,0.90,0.86,NO
160,Pontiac Municipal Airport,Oakland County International Airport,Illinois,0.58,0.86,0.88,NO
1133,Scottsville,"Scottsville, Kentucky",Kentucky,0.48,0.00,0.42,OK
2864,Scottsville,"Scottsville, California",California,0.52,0.00,0.46,OK
4891,Scottsville,"Scottsville, Kansas",Kansas,0.42,0.00,0.42,OK


#### Check for slavic languages 

In [90]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name_country']=='Ukraine')& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'

In [91]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name_country']=='Bulgaria')& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'

In [92]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name_country']=='Poland')& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'

In [93]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name_country']=='Slovenia')& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name_country']=='Croatia')& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name_country']=='Moldova')& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'

#### Check for Greek languages 

In [94]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='ΝΕΟΙ ΕΠΙΒΑΤΑΙ') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='ΙΕΡΙΣΣΟΣ') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='ΠΑΡΑΛΙΑ') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='ΚΑΛΛΙΘΕΑ') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='ΥΔΡΑ') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='ΙΟΣ') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='ΣΤΑΥΡΟΣ') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='ΑΡΜΑ') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='ΒΑΘΥ') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='ΑΡΙΑ') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Amfithea') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Káto Ília') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Ilioupoli Athinon') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Kynosargous') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Dourgouti') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Katsipodi') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Mets') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Λευκωσία') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Λεμεσός') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='ΑΓΙΟΝ ΠΝΕΥΜΑ') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'


In [95]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name_country']=='Greece')& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name_country']=='Cyprus')& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'

#### Check for Spanish language

In [96]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Penilla, La') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Revilla') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='La Guardia (Santa María)') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Baies, les') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Port de Llançà, el') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Apatamonasterio') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Caraquiz') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Grau de Castelló') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Alberche del Caudillo') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Penilla, La') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Bay Le Moine') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Gallarta') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Real de Montroi') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Tanque (El)') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Aeropuerto de la Palma') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Aeropuerto de Santiago') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Aerodromo Provincial') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Illes Balears') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Sanchinarro') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Pinares de Venecia') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='La Milagrosa') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Valle Los Galanes') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name']=='Illes Balears') & (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'NO'

In [97]:
unique_concordances_fix_maybe.ix[(unique_concordances_fix_maybe['name_country']=='Spain')& (unique_concordances_fix_maybe['spell_check']=='maybe'), 'spell_check'] = 'OK'

#### Check for Districts, Municipalities or Disambiguation 

In [98]:
unique_concordances_fix_maybe_2=unique_concordances_fix_maybe

In [99]:
unique_concordances_fix_maybe_3=[]
for index, row in unique_concordances_fix_maybe_2.iterrows():
    if str(row['name']).find("District")!=-1:
        row['spell_check'] = 'OK'
    elif str(row['name']).find("Municipality")!=-1:
        row['spell_check'] = 'OK'
    elif str(row['wk:page']).find("disambiguation")!=-1:
        row['spell_check'] = 'NO'
    else:
        pass
    unique_concordances_fix_maybe_3.append(row)
unique_concordances_fix_maybe_3=pd.DataFrame(unique_concordances_fix_maybe_3)


### Import some manual edits 

In [100]:
manually=read_data("..\Analyzed_data\\manual_titles.csv")

In [101]:
unique_concordances_fix_maybe_4=unique_concordances_fix_maybe_3.join(manually.set_index(['wof:id']), on='wof:id', how = 'left')

In [102]:
unique_concordances_fix_maybe_4['spell_check']=np.where(unique_concordances_fix_maybe_4['status']=='OK',unique_concordances_fix_maybe_4['status'],unique_concordances_fix_maybe_4['spell_check'])
unique_concordances_fix_maybe_4.ix[(unique_concordances_fix_maybe_4['wof:id']==102078709), 'spell_check'] = 'NO'
unique_concordances_fix_maybe_4.ix[(unique_concordances_fix_maybe_4['wof:id']==420540203), 'spell_check'] = 'NO'

#### Add placetype from SPARQL and check if right placetype 

In [103]:
all_wiki_data=read_data('..\Analyzed_data\\SPARQL_all.csv')

In [104]:
all_wiki_data=all_wiki_data[['wd:id','place']]

In [105]:
unique_concordances_fix_maybe_5=unique_concordances_fix_maybe_4[['name','wk:page','wd:id','placetype','spell_check','wof:id','leve_dist','leve_dist_actual', 'wk_page_id' ,'wordcount']]

In [106]:
all_wiki_data=find_unique(all_wiki_data, 'wd:id')

In [107]:
unique_concordances_fix_maybe_6=unique_concordances_fix_maybe_5.join(all_wiki_data.set_index(['wd:id']), on='wd:id', how='left')

In [108]:
unique_concordances_fix_maybe_6_dups = unique_concordances_fix_maybe_6[(unique_concordances_fix_maybe_6.duplicated('wd:id', keep=False)) & (unique_concordances_fix_maybe_6['place'].notnull())]

In [109]:
unique_concordances_fix_dups=[]
for index , row in unique_concordances_fix_maybe_6_dups.iterrows():
    if str(row['placetype']) !=' empire':
        if str(row['placetype']) != str(row['place']):
            row['check'] = 'NO'
        else:
            pass
    unique_concordances_fix_dups.append(row)
    
unique_concordances_fix_dups=pd.DataFrame(unique_concordances_fix_dups)        

In [110]:
unique_concordances_fix_dups=unique_concordances_fix_dups[['wof:id','check']]

In [111]:
unique_concordances_fix_maybe_7=unique_concordances_fix_maybe_6.join(unique_concordances_fix_dups.set_index(['wof:id']), on='wof:id', how='left')

In [112]:
unique_concordances_fix_maybe_7['spell_check']=np.where(unique_concordances_fix_maybe_7['check'].notnull(),unique_concordances_fix_maybe_7['check'],unique_concordances_fix_maybe_7['spell_check'])

In [113]:
titles_OK = unique_concordances_fix_maybe_7[unique_concordances_fix_maybe_7['spell_check']=='OK']

#### If multiple entries have the same title set it to the highest placetype 

In [114]:
titles_OK_rank=[]
for index, row in titles_OK.iterrows():
    if row['placetype'] == 'continent':
        row['place_rank'] = 1
    elif row['placetype'] == 'country':
        row['place_rank'] = 2
    elif row['placetype'] == 'macroregion':
        row['place_rank'] = 3
    elif row['placetype'] == 'region':
        row['place_rank'] = 4
    elif row['placetype'] == 'macrocounty':
        row['place_rank'] = 5
    elif row['placetype'] == 'county':
        row['place_rank'] = 6
    elif row['placetype'] == 'locality':
        row['place_rank'] = 7
    elif row['placetype'] == 'borough':
        row['place_rank'] = 8
    elif row['placetype'] == 'macrohood':
        row['place_rank'] = 9
    elif row['placetype'] == 'neighborhood':
        row['place_rank'] = 10
    elif row['placetype'] == 'microhood':
        row['place_rank'] = 11
    elif row['placetype'] == 'campus':
        row['place_rank'] = 12
    titles_OK_rank.append(row)
titles_OK_rank = pd.DataFrame(titles_OK_rank)

In [115]:
titles_OK_dups=titles_OK_rank[titles_OK_rank.duplicated('wk:page', keep=False)]

In [116]:
titles_grouped=titles_OK_dups.groupby('wk:page')

In [117]:
all_names= list(titles_OK_dups['wk:page'].unique())

In [118]:
index = np.arange(0)
problematic = pd.DataFrame(index=index, columns=titles_OK_dups.columns)
okay = pd.DataFrame(index=index, columns=titles_OK_dups.columns)

In [119]:
for name in all_names:
    new_group=titles_grouped.get_group(name)
    placetypes = list(new_group['placetype'].unique())
    if len(placetypes)>1:
        rank = min(new_group['place_rank'])
        new_group['status'] = np.where(new_group['place_rank']==rank,'OK','NO')
    else:
        new_group['place_rank']='OK'
    problematic = problematic.append(new_group)
        

In [120]:
problematic['spell_check']=np.where(problematic['status'].notnull(),problematic['status'],problematic['spell_check'])

In [121]:
problematic_fix = problematic[['wof:id' , 'spell_check']]

In [122]:
unique_concordances_fix_maybe_8 =unique_concordances_fix_maybe_7.join(problematic_fix.set_index(['wof:id']), on = 'wof:id' , rsuffix='_right')

In [123]:
unique_concordances_fix_maybe_8['spell_check']=np.where(unique_concordances_fix_maybe_8['spell_check_right'].notnull(),unique_concordances_fix_maybe_8['spell_check_right'],unique_concordances_fix_maybe_8['spell_check'])

In [124]:
unique_concordances_fix_maybe_9 = unique_concordances_fix_maybe_8.drop(['leve_dist','leve_dist_actual','place','check','spell_check_right' ],1)

In [125]:
unique_concordances_fix_maybe_9.to_csv("..\Analyzed_data\\final_titles_ids_all_status.csv")

In [3]:
unique_concordances_fix_maybe_9 = read_data("..\Analyzed_data\\final_titles_ids_all_status.csv")

In [7]:
unique_concordances_fix_maybe_9[unique_concordances_fix_maybe_9['spell_check']!='OK']['name']

22                               Brooklyn
23                                  Bronx
24                              Manhattan
26                                 Queens
29                  Dhaka Tejgaon Airport
32                       Rajshahi Airport
33                     Chittagong Airport
34                  Osmany Sylhet Airport
35        Zia International Airport Dhaka
37                      Tinfouchy Airport
38                       Oued Mia Airport
42                      El Mellah Airport
44                       Bou Sfer Airport
48          El Abiodh Sidi Cheikh Airport
49                       Laghouat Airport
52                     Bou Chekif Airport
53                         Ouakda Airport
58                         Bechar Airport
60                    Tamanrasset Airport
64                       Hamaguir Airport
65                      In Amguel Airport
67                          Setif Airport
71                        Tebessa Airport
73                          Tiska 

#### Print for blog 

In [127]:
test2=unique_concordances_fix_maybe_9[((unique_concordances_fix_maybe_9['name']=='South America')|
                                   (unique_concordances_fix_maybe_9['wk:page']=='Warkworth')|
                                   (unique_concordances_fix_maybe_9['name']=='Bronx')|
                                   (unique_concordances_fix_maybe_9['name']=='Rajshahi Airport')|
                                   (unique_concordances_fix_maybe_9['name']=='Tafaraoui Airport')|
                                   (unique_concordances_fix_maybe_9['name']=='Laghouat Airport')|
                                   (unique_concordances_fix_maybe_9['name']=='Sky Harbor Residential Airpark')|
                                   (unique_concordances_fix_maybe_9['name']=='Marked Tree Municipal Airport')|
                                   (unique_concordances_fix_maybe_9['name']=='Junction Airport')|
                                   (unique_concordances_fix_maybe_9['name']=='La Fonda Ranch Airport')|
                                   (unique_concordances_fix_maybe_9['name']=='Woods of Eden Rock')|
                                   (unique_concordances_fix_maybe_9['name']=='Woods of Eden Rock'))]
test3 = test2[test2['placetype']!='borough']
test3['wof:name']=test3['name']
out1=test3[['wof:name','wk:page']]
out1.to_html(open('..\..\Blog_post_wikipedia\images\\original.html', 'w'))
out1

,wof:name,wk:page
3,South America,South America
32,Rajshahi Airport,List of airports in Bangladesh
39,Tafaraoui Airport,Oran Tafraoui Airport
49,Laghouat Airport,L'Mekrareg Airport
257,Sky Harbor Residential Airpark,Hernando Village Airpark
517,Marked Tree Municipal Airport,Arkansas Highway 308
905,Junction Airport,Grand Junction Regional Airport
1279,La Fonda Ranch Airport,Once Upon a Time in the West
73424,Woods of Eden Rock,Rock music
154819,Bronx,The Bronx


In [128]:
test3 = test2[test2['placetype']!='borough']
test3['wof:name']=test3['name']
test3['correct']=test3['spell_check']
test3.ix[(test3['name']=='Rajshahi Airport'), 'correct'] = 'NO'
test3.ix[(test3['name']=='La Fonda Ranch Airport'), 'correct'] = 'NO'
final_out = test3[['wof:name','wk:page', 'correct']]
final_out.to_html(open('..\..\Blog_post_wikipedia\images\\final.html', 'w'))
final_out

,wof:name,wk:page,correct
3,South America,South America,OK
32,Rajshahi Airport,List of airports in Bangladesh,NO
39,Tafaraoui Airport,Oran Tafraoui Airport,OK
49,Laghouat Airport,L'Mekrareg Airport,maybe
257,Sky Harbor Residential Airpark,Hernando Village Airpark,maybe
517,Marked Tree Municipal Airport,Arkansas Highway 308,NO
905,Junction Airport,Grand Junction Regional Airport,maybe
1279,La Fonda Ranch Airport,Once Upon a Time in the West,NO
73424,Woods of Eden Rock,Rock music,NO
154819,Bronx,The Bronx,OK


 ### Save the names that were wrong

In [129]:
names_do_not_want=list(unique_concordances_fix_maybe_9[unique_concordances_fix_maybe_9['spell_check']!='OK']['name'])

In [130]:
with open('names_do_not_want', 'wb') as f:
    pickle.dump(names_do_not_want, f)

### Save final clean table for wikidata ids

In [131]:
final_WD_ID = unique_concordances_fix_maybe_9[(unique_concordances_fix_maybe_9['wd:id'].notnull())&(unique_concordances_fix_maybe_9['spell_check']=='OK')]
final_WD_ID = final_WD_ID[['wof:id' , 'wd:id']]

In [132]:
final_WD_ID.to_csv("..\OUTPUT_wd_id\\final_wd_ids_clean.csv")

###  Generate Urls

In [133]:
final_titles = unique_concordances_fix_maybe_9[(unique_concordances_fix_maybe_9['wk:page'].notnull())&(unique_concordances_fix_maybe_9['spell_check']=='OK')]
final_titles['url'] = 'https://en.wikipedia.org/wiki/'+final_titles['wk:page']
final_titles = final_titles[['wof:id' , 'wk:page', 'url']]

In [134]:
final_titles.to_csv("..\OUTPUTS_titles_word_count\\final_titles_clean.csv")

### Print placetypes for blog

In [135]:
names_wrong = unique_concordances_fix_maybe_9[((unique_concordances_fix_maybe_9['wk:page']=='1974 Super Outbreak')|(unique_concordances_fix_maybe_9['name']=='City of Friendship Airport')|
                                 (unique_concordances_fix_maybe_9['name']=='Flying Crown Airport')|(unique_concordances_fix_maybe_9['name']=='Amado Bahia')|
                                 (unique_concordances_fix_maybe_9['wk:page']=='Knapp Gardens Plat')|(unique_concordances_fix_maybe_9['wk:page']=='Pshizov')|
                                 (unique_concordances_fix_maybe_9['name']=='Dhaka Tejgaon Airport')|(unique_concordances_fix_maybe_9['name']=='Grafs First'))]
names_wrong.ix[(names_wrong['name']=='Amado Bahia'), 'spell_check'] = 'NO'
names_wrong['wof:name']=names_wrong['name']
names_wrong['correct']=names_wrong['spell_check']
airports = names_wrong[['wof:name','wk:page', 'correct']]
import sys 
reload(sys) 
sys.setdefaultencoding('utf-8')
airports.to_html(open('..\..\Blog_post_wikipedia\images\\lists_music.html', 'w'))


In [136]:
airports

,wof:name,wk:page,correct
29,Dhaka Tejgaon Airport,List of airports in Bangladesh,NO
123,Flying Crown Airport,List of airports in Canada (A–B),NO
164,Wawasee Airport,1974 Super Outbreak,NO
1345,City of Friendship Airport,National Airlines (1934–1980),NO
40303,Amado Bahia,"Timeline of Salvador, Bahia",NO
78901,Grafs First,Rolf Graf (musician),NO


In [137]:
district_ok= unique_concordances_fix_maybe_9[((unique_concordances_fix_maybe_9['wk:page'].str.contains('Dolní Újezd'))|
                                             (unique_concordances_fix_maybe_9['wk:page'].str.contains('Ribnica'))|
                                             (unique_concordances_fix_maybe_9['name'].str.contains('Tinfouchy Airport'))|
                                             (unique_concordances_fix_maybe_9['name']=='Logan County Airport'))]
district_ok=district_ok[(district_ok['placetype']!='region')]
district_ok=district_ok[(district_ok['wk:page']!='Ribnica na Pohorju')]
district_ok=district_ok[district_ok['wof:id']!=102522829]
district_ok.ix[(district_ok['name']=='Tinfouchy Airport'), 'spell_check'] = 'NO'
district_ok['wof:name']=district_ok['name']
district_ok['correct']=district_ok['spell_check']
district_ok=district_ok[['wof:name','wk:page', 'correct']]
district_ok.to_html(open('..\..\Blog_post_wikipedia\images\\airports.html', 'w'))
district_ok

,wof:name,wk:page,correct
37,Tinfouchy Airport,Tinfouchy,NO
2317,Logan County Airport,Logan Airport (disambiguation),NO
13206,Dolní Újezd,Dolní Újezd (Svitavy District),OK
229925,Ribnica,"Ribnica, Ribnica",OK


In [138]:
placetype = unique_concordances_fix_maybe_9[unique_concordances_fix_maybe_9['name']=='China']
placetype = placetype[placetype['placetype']!='empire']
placetype['wof:name']=placetype['name']
placetype['correct']=placetype['spell_check']
placetype_out = placetype[['wof:name', 'wk:page', 'placetype','correct']]
placetype_out.to_html(open('..\..\Blog_post_wikipedia\images\\placetype.html', 'w'))
placetype_out


,wof:name,wk:page,placetype,correct
12,China,China,country,OK
153054,China,China,county,NO
184843,China,"China, Nuevo León",locality,OK
208292,China,"China, Texas",locality,OK


In [139]:
wd_ids = unique_concordances_fix_maybe_9[(unique_concordances_fix_maybe_9['name']=='Georgia')|
                                        (unique_concordances_fix_maybe_9['name']=='Somalia')|
                                        (unique_concordances_fix_maybe_9['name']=='Kazakhstan')|
                                        (unique_concordances_fix_maybe_9['wk:page']=='Staten Island')|
                                        (unique_concordances_fix_maybe_9['name']=='San Marino')|
                                        (unique_concordances_fix_maybe_9['name']=='Brooks Air Force Base')|
                                        (unique_concordances_fix_maybe_9['name']=='Los Angeles Air Force Base')|
                                        (unique_concordances_fix_maybe_9['name']=='Asia')]
wd_ids=wd_ids[(wd_ids['wk:page']!='San Marino, California')]
wd_ids=wd_ids[(wd_ids['wk:page']!='Georgia (country)')]
wd_ids=wd_ids[(wd_ids['wk:page']!='City of San Marino')]
wd_ids['wof:name']=wd_ids['name']
wd_ids=find_unique(wd_ids,'wk:page')
wd_ids = wd_ids[['wof:id', 'wof:name', 'wk:page', 'wd:id']]
wd_ids.to_html(open('..\..\Blog_post_wikipedia\images\\wd_ids.html', 'w'))
wd_ids

,wof:id,wof:name,wk:page,wd:id
0,102191569,Asia,Asia,Q48
17,421166797,Georgia,Georgia,Q4962
18,421180189,Somalia,Somalia,Q1045
19,421185849,Kazakhstan,Kazakhstan,Q232
21,421202109,San Marino,San Marino,Q238
25,421205775,Staten Island,Staten Island,Q18432
100,102521867,Brooks Air Force Base,Brooks Air Force Base,Q4974968
103,102521877,Los Angeles Air Force Base,Los Angeles Air Force Base,Q6681941
